# Stefan's Law: Simulation
This is R code. Refer to [the installation manual](https://cran.r-project.org/web/packages/IRkernel/readme/README.html) for instructions on how to execute it.

## Variables to be changed

In [ ]:
distances <- c(0.3, 0.4, 0.6, 0.8, 1)               # Distances emitter-thermopile (m)
T_ext <- 293.15                                     # External temperature (K)
temperatures <- c(293.15, 333.15, 363.15, 395.15)   # Emmiter temperatures (K)
isFiltered <- TRUE                                  # Is the thermopile filtered? (Bool)
times <- 1:50                                       # Measurement points (default: 1:50 - once every second for 50 seconds)

### Other variables

In [ ]:
alpha <- 30         # Datasheet: sensitivity 
r <- 0.025          # Red book
d0 <- 0.04          # Red book
sigma <- 5.675e-8   # Stefan-Boltzmann constant
KC <- 0.1           # Constant (explained later)
noise_sd <- 0.05    # Datasheet

if (!isFiltered) {
    alpha <- 1.1*alpha
}

## Function of temperature wrt time

Parameters:
- `t`: time to be evaluated (continuous variable, geq 0)
- `PK`: constant. Proportional to the power received by the thermopile
- `KC`: constant. Proportional to the inverse of the heat capacity for the thermopile
- `T_ext`: external temperature

In [16]:
T_func <- function(t, PK, KC, T_ext) {
    PK * (1-exp(-KC*t)) + T_ext
}

## Generating the functions for different conditions

In [32]:
for (d in distances){
    for (T in temperatures){
        # Figure out the parameters
        constant <- alpha*(r/(r+d+d0))^2
        PK = T-T_ext

        # Define the function
        func_name <- paste0("DeltaV_d", d, "_T", T)
        original_f <- function(x) constant*T_func(x, PK, KC, T_ext)
        assign(func_name, f)

        # Adding noise
        noisy_func_name <- paste0("NoisyDeltaV_d", d, "_T", T)
        f <- function(x) {
            original_val <- original_f(x)
            noise <- rnorm(n=1, mean=1, sd=0.05)*original_val
            return(noise)
        }
        assign(noisy_func_name, f)

        # Exporting to csv
        df <- data.frame("Time"=times, "Delta V"=f(times))
        filename <- paste0("output_d", d, "_T", T, ".csv")
        write.csv(df, filename, row.names=FALSE)

    }
}